# Codex: 29 Markers on 5,000 Murine Spleen Cells
Data obtained and re-processed from http://welikesharingdata.blob.core.windows.net/forshare/index.html.

In [1]:
import pandas as pd
df = {}

In [2]:
from clustergrammer2 import net

import ipywidgets as widgets
import numpy as np
from bqplot import pyplot as plt
import bqplot

from copy import deepcopy
from glob import glob
from scipy.spatial.distance import pdist, squareform
from scipy.spatial import Voronoi

import warnings
warnings.filterwarnings('ignore')

>> clustergrammer2 backend version 0.5.1


In [3]:
net.load_file('codex_data/4-tile_5k-cells_expression-cat.txt')
df['tile-neighbor'] = net.export_df()

(29, 5018)

In [4]:
net.load_file('codex_data/4-tile_5k-cells_location-cat.txt')
df['tile-loc'] = net.export_df()

(5146, 2)

In [5]:
cat_colors = net.load_json_to_dict('codex_data/cat_colors.json')

In [6]:
net.load_df(df['tile-neighbor'])
net.set_cat_colors(axis='col', cat_colors=cat_colors, cat_index=1, cat_title='Cell Type')
net.set_cat_colors(axis='col', cat_colors=cat_colors, cat_index=2, cat_title='Neighbor')

In [7]:
def cat_highlight(inst_value):

    if inst_value != 'reset_cats':
        
        inst_cat = inst_value.split(': ')[1]

        list_opacities = []

        for inst_label in region_labels:
            inst_opacity = 0.25
            if inst_label == inst_cat:
                inst_opacity = 1
            list_opacities.append(inst_opacity)

    else:
        list_opacities = [1 for x in region_labels]
        patch.opacities = list_opacities

    patch.opacities = list_opacities

In [8]:
df['tile-lookup'] = deepcopy(df['tile-neighbor'])
df['tile-lookup'].columns = [x[0] for x in df['tile-lookup'].columns]

In [9]:
def on_value_change(change):
    
    if change['new'] == 'null':
        cat_highlight('reset_cats')            
    else: 
        # mousing over category
        if 'cat-' in change['new']:
            inst_cat = change['new'].split(' -> ')[1]
            cat_highlight(inst_cat)       
            
        # mousing over marker
        else:
            inst_marker = change['new'].split(' -> ')[1]
            ser_opacity = df['tile-lookup'].loc[inst_marker]
            
            
            # the patches are not in the same order as the data in the dataframe
            #list_opacities = [float(x/ser_opacity.max()) for x in list(ser_opacity.get_values())]
            
            # loop up opacities
            list_opacities = []
            rows = ser_opacity.index.tolist()
            for inst_name in keep_names:
                if inst_name in rows:
                    list_opacities.append( float(ser_opacity[inst_name]/ser_opacity.max())  )
                else:
                    list_opacities.append(0)
                
            patch.opacities = list_opacities


In [10]:
net.load_df(df['tile-neighbor'])
net.random_sample(axis='col', num_samples=3000, random_state=99)
net.normalize(axis='row', norm_type='zscore')
net.clip(-5,5)
net.widget()
net.widget_instance.observe(on_value_change, names='value')
# tmp = net.widget()

In [11]:
vor = Voronoi(df['tile-loc'])

point_list = df['tile-loc'].index.tolist()
point_names = [x[0] for x in point_list]
cat_names   = [x[1].split(': ')[1] for x in point_list]

patch_data = {}
patch_data['x'] = []
patch_data['y'] = []
patch_data['colors'] = []
region_labels = []

keep_names = []

region_point_dict = {}
for point_index in range(vor.point_region.shape[0]):
    region_index = vor.point_region[point_index]
    region_point_dict[region_index] = point_index

for region_index in range(len(vor.regions)):
    
    inst_region = vor.regions[region_index]

    if -1 not in inst_region and len(inst_region) > 0:

        point_index = region_point_dict[region_index]
        point_cat = cat_names[point_index]
        point_name = point_names[point_index]
        region_labels.append(point_cat)
        
        keep_names.append(point_name)
        
        # save cat_colors
        inst_color = cat_colors[point_cat]
        patch_data['colors'].append(inst_color)
        
        x_list = []
        y_list = []
        for inst_vertex in inst_region:
            inst_pos = vor.vertices[inst_vertex]
            x_list.append(inst_pos[0])
            y_list.append(inst_pos[1])
            
        patch_data['x'].append(x_list)
        patch_data['y'].append(y_list)    

In [40]:
x_dim = 1200
y_dim = 1000

def_tt = bqplot.Tooltip(fields=['name'], formats=[''])

fig = plt.figure(animation_duration=1000)
patch = plt.plot([], [], 
                 fill='inside',
                 fill_colors=patch_data['colors'],
                 stroke_width=1,
                 close_path=True,
                 labels=region_labels,
                 tooltip=def_tt,
                 axes_options={'x': {'visible': False}, 'y': {'visible': False}},
                )

scatter = plt.scatter(df['tile-loc']['X.X'], 
                      df['tile-loc']['Y.Y'],
#                       tooltip=def_tt, 
                      names=point_names,
                      display_names=False, default_size=2)


top_margin = 275
inst_width = 800
inst_height = inst_width/1.15 + top_margin
fig.layout.min_height = str(inst_height) + 'px'
fig.layout.min_width  = str(inst_width) + 'px'

patch.x = patch_data['x']
patch.y = patch_data['y']

plt.xlim(0, 2.0*x_dim)
plt.ylim(0, 2.0*y_dim)
fig.title = 'Codex Cell Locations'
fig.fig_margin = {'top': top_margin, 'bottom': 5, 'left': 5, 'right': 5}

# fig.margin {top=10, bottom=10, left=10, right=10}

# plt.show()

In [35]:
from ipywidgets import HBox
HBox([fig, net.widget_instance])